# NLE Assessed Coursework 3: Question 2

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 2 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [14]:
candidateno=184521 #this MUST be updated to your candidate number so that you get a unique data sample


In [52]:
#preliminary imports
import sys
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from sussex_nltk.corpus_readers import ReutersCorpusReader
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import math
from nltk.corpus import wordnet as wn

from collections import Counter
def normalise(tokenlist):
    tokenlist=[token.lower() for token in tokenlist]
    tokenlist=["NUM" if token.isdigit() else token for token in tokenlist]
    tokenlist=["Nth" if (token.endswith(("nd","st","th")) and token[:-2].isdigit()) else token for token in tokenlist]
    tokenlist=["NUM" if re.search("^[+-]?[0-9]+\.[0-9]",token) else token for token in tokenlist]
    return tokenlist



## Question 2: Distributional Semantics (25 marks)


In [53]:
rcr = ReutersCorpusReader().finance()
rcr.enumerate_sents()

1113359

In [54]:
random.seed(candidateno)  
samplesize=2000
iterations =100
sentences=[]
for i in range(0,iterations):
    sentences+=[normalise(sent) for sent in rcr.sample_sents(samplesize=samplesize)]
    print("Completed {}%".format(i))
print("Completed 100%")

Completed 0%
Completed 1%
Completed 2%
Completed 3%
Completed 4%
Completed 5%
Completed 6%
Completed 7%
Completed 8%
Completed 9%
Completed 10%
Completed 11%
Completed 12%
Completed 13%
Completed 14%
Completed 15%
Completed 16%
Completed 17%
Completed 18%
Completed 19%
Completed 20%
Completed 21%
Completed 22%
Completed 23%
Completed 24%
Completed 25%
Completed 26%
Completed 27%
Completed 28%
Completed 29%
Completed 30%
Completed 31%
Completed 32%
Completed 33%
Completed 34%
Completed 35%
Completed 36%
Completed 37%
Completed 38%
Completed 39%
Completed 40%
Completed 41%
Completed 42%
Completed 43%
Completed 44%
Completed 45%
Completed 46%
Completed 47%
Completed 48%
Completed 49%
Completed 50%
Completed 51%
Completed 52%
Completed 53%
Completed 54%
Completed 55%
Completed 56%
Completed 57%
Completed 58%
Completed 59%
Completed 60%
Completed 61%
Completed 62%
Completed 63%
Completed 64%
Completed 65%
Completed 66%
Completed 67%
Completed 68%
Completed 69%
Completed 70%
Completed 71%
Co

In [55]:
def generate_features(sentences,window=1):#given the sentences array as generated above and _____
    mydict={} #create Dictionary
    for sentence in sentences: #takes each sentence from the given sentences array
        for i,token in enumerate(sentence): #iterates through every word in a the selected sentence
            current=mydict.get(token,{}) #retrieves a set from mydict and applies it to the variable 'current'
            features=sentence[max(0,i-window):i]+sentence[i+1:i+window+1] #variable 'features' is created and is assigned
                                                                          #the words from a sentence
            for feature in features:#iterates through every feature in features
                current[feature]=current.get(feature,0)+1 #the selected "feature" is then used to get a token and assign it 
                                                          #to current
            mydict[token]=current #the current set is then added to 'mydict' with the key being the current 'token'
    return mydict # dictionary 'mydict' is then returned

a) **Run** `generate_features(sentences[:5])`.  With reference to the code and the specific examples, **explain** how the output was generated. \[4 marks\]

In [70]:
generate_features(sentences[:5])

{'amount': {'issuer/type': 1},
 'issuer/type': {'amount': 1, 'sale': 1},
 'sale': {'issuer/type': 1, 'date': 1},
 'date': {'sale': 1},
 'many': {'german': 1},
 'german': {'many': 1, 'officials': 1},
 'officials': {'german': 1, 'now': 1},
 'now': {'officials': 1, 'say': 1},
 'say': {'now': 1, 'the': 1},
 'the': {'say': 1, 'correction': 1, 'that': 1, 'bank': 1},
 'correction': {'the': 1, 'is': 1},
 'is': {'correction': 1, 'complete': 1},
 'complete': {'is': 1, '.': 1},
 '.': {'complete': 1, 'hold': 1},
 'mario': {'angastiniotis': 1},
 'angastiniotis': {'mario': 1, ',': 1},
 ',': {'angastiniotis': 1, 'economist': 2, 'mms': 1},
 'economist': {',': 2},
 'mms': {',': 1, 'international': 1},
 'international': {'mms': 1, ':': 1},
 ':': {'international': 1, '``': 1},
 '``': {':': 1, 'it': 1},
 'it': {'``': 1, 'means': 1},
 'means': {'it': 1, 'that': 1},
 'that': {'means': 1, 'the': 1},
 'bank': {'the': 1, 'of': 1},
 'of': {'bank': 1, 'canada': 1},
 'canada': {'of': 1, 'still': 1},
 'still': {'c

The 'generate_features' function begins with it being given the sentences array, which was generated in the previous cell and contains a sample of 100 sentences picked using the seed being my candidate number, but only the first 5 sentences of the 100 sentence array. The function is also given the variable 'window' which is assigned the value of 1.
After this it initialises a dictionary, 'mydict', before iterating through the five given sentences with a for loop before enumerating each word of the sentence allowing it to be iterated through using its value 'i' and the word 'token'.
The function then assigns the variable 'current' with a blank set from 'mydict' with the current word, 'token' being its key along with assigning the variable 'features' with the  a set of two words either side of the current token being at positions 'i-1' and 'i+1' with 'i' being the tokens position.
The variable 'features' is then iterated through feature by feature and each feature as a key for that features previous and next tokens allowing all features of a sentence to be put into a single variable as a dictionary. This dictionary can then be put into the dictionary 'mydict' which becomes a dictionary of all of the sentences features. A feature being a tokens that are either side of the token in its sentence, however if a token appears more than once then this means it has multiple features and therefores shall have multiple words following its key in the dictionary. Each feature is also given a counter to show how many times it is a feature of the given word.

b) Write code and **find** the 100 most frequently occurring words that
* are in your sample; AND
* have at least one noun sense according to WordNet
\[ 4 marks\]

In [56]:
def getNouns(sentences):
    #initialise noun array
    sym={}
    nouns=[]
    temp=''
    #iterate through each word in the sentence and each sentence in the given sentences
    for sentence in sentences:
        for word in sentence:
            #add them to nouns array if a noun
            if(len(wn.synsets(word,"n"))>0):
                nouns.append(word)
    return nouns

nouns= getNouns(sentences)

In [57]:
def hundredNouns(sentences,n):
    #remove all punctuation
    #count how often each word appears
    c=Counter(n)
    #get top 100
    topHundred=c.most_common(100)
    #return top 100
    return (topHundred)

In [59]:
hundredNouns(sentences,nouns)

[('in', 56047),
 ('a', 49052),
 ('percent', 17923),
 ('it', 15516),
 ('be', 14624),
 ('pct', 14508),
 ('at', 13930),
 ('was', 13406),
 ('will', 11124),
 ('he', 11011),
 ('as', 10887),
 ('bank', 9319),
 ('year', 8929),
 ('million', 8820),
 ('has', 8583),
 ('have', 8575),
 ('may', 8507),
 ('are', 8485),
 ('its', 8387),
 ('government', 8305),
 ('an', 7949),
 ('billion', 7128),
 ('tax', 5776),
 ('uk', 5693),
 ('trade', 5554),
 ('rate', 5430),
 ('budget', 4993),
 ('more', 4833),
 ('growth', 4668),
 ('last', 4622),
 ('u.s.', 4447),
 ('first', 4431),
 ('central', 4259),
 ('price', 4082),
 ('market', 4026),
 ('june', 3989),
 ('newsroom', 3986),
 ('over', 3933),
 ('or', 3911),
 ('union', 3833),
 ('there', 3809),
 ('inflation', 3773),
 ('state', 3753),
 ('bonds', 3682),
 ('rates', 3479),
 ('one', 3410),
 ('minister', 3404),
 ('i', 3404),
 ('economy', 3365),
 ('week', 3357),
 ('no', 3357),
 ('can', 3324),
 ('prices', 3296),
 ('april', 3259),
 ('currency', 3220),
 ('interest', 3138),
 ('two', 3137

c) Write code to create distributional vector representations of each word in the corpus with a parameter to specify context window size.   \[5 marks \]

In [75]:
def vectorRep(sentences,window):
    temp=[]
    for tokens in sentences:
        for i,word in enumerate(tokens):
            temp.append((word,tokens[max(0,i-window):i]+tokens[i+1:i+window+1]))
    return temp;

In [86]:
vector=vectorRep(sentences,1)

In [87]:
(pd.DataFrame(vector,columns=["Word","Distrubution"]))

,Word,Distrubution
0,the,[increase]
1,increase,"[the, followed]"
2,followed,"[increase, a]"
3,a,"[followed, revised]"
4,revised,"[a, NUM]"
5,NUM,"[revised, percent]"
6,percent,"[NUM, drop]"
7,drop,"[percent, in]"
8,in,"[drop, may]"
9,may,"[in, ,]"


d) Plan and carry out an investigation into the correlation between semantic similarity according to the WordNet path similarity measure and distributional similarity with different context window sizes.  You should make sure that you include a graph of how correlation varies with context window size and that you discuss your results.  \[12 marks\]